# Conditional Counterfactual Density Estimation Demo

This notebook demonstrates the use of **Conditional Counterfactual Mean Embedding (CCME)** for estimating conditional distributions of counterfactual outcomes.

## Overview

We'll use the **IHDP (Infant Health and Development Program)** dataset to:
1. Estimate conditional counterfactual densities $p(Y^1 | V)$
2. Compare different estimators (ridge, df, nk)
3. Evaluate Doubly Robust (dr) vs One-Step (onestep) methods
4. Visualize heterogeneous treatment effects beyond means

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from omegaconf import OmegaConf
import ipywidgets as widgets
from IPython.display import display, clear_output

from ccme import CCDEstimator

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports successful!")

## Load IHDP Dataset

The IHDP dataset contains data from a randomized experiment evaluating the effect of intensive childcare on cognitive test scores.

- **Treatment (A)**: Participation in intensive childcare program (0 or 1)
- **Outcome (Y)**: Cognitive test score (continuous)
- **Covariates (X)**: Mother's age, education, birth weight, etc.

In [ ]:
def load_ihdp_data():
    """
    Load IHDP dataset. 
    For this demo, we'll generate a semi-synthetic version based on IHDP characteristics.
    In practice, you can download the real IHDP data from causal benchmarking repositories.
    """
    np.random.seed(42)
    n = 747  # Approximate IHDP sample size
    
    # Generate covariates (simplified IHDP structure)
    # X1: mother's age (centered), X2: mother's education, X3: birth weight, etc.
    X = np.random.randn(n, 10)
    
    # Treatment assignment (mimicking IHDP's ~2:1 control:treatment ratio)
    propensity = 1 / (1 + np.exp(-0.5 * (X[:, 0] + X[:, 1])))
    A = np.random.binomial(1, propensity)
    
    # Potential outcomes with heterogeneous effects
    Y0 = 80 + 10 * X[:, 0] + 5 * X[:, 1] - 3 * X[:, 2] + np.random.randn(n) * 5
    treatment_effect = 10 + 5 * X[:, 0] - 2 * X[:, 1]  # Heterogeneous
    Y1 = Y0 + treatment_effect + np.random.randn(n) * 5
    
    # Observed outcome
    Y = A * Y1 + (1 - A) * Y0
    
    return X, Y[:, None], A, Y0, Y1

# Load data
X, Y, A, Y0_true, Y1_true = load_ihdp_data()
n_samples = len(Y)

print(f"Dataset loaded: {n_samples} samples")
print(f"  - Treated: {A.sum()} ({A.mean()*100:.1f}%)")
print(f"  - Control: {(1-A).sum()} ({(1-A.mean())*100:.1f}%)")
print(f"  - Covariates: {X.shape[1]} dimensions")
print(f"  - Outcome range: [{Y.min():.1f}, {Y.max():.1f}]")

## Data Exploration

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Outcome distribution by treatment
axes[0].hist(Y[A==0], bins=30, alpha=0.6, label='Control (A=0)', density=True)
axes[0].hist(Y[A==1], bins=30, alpha=0.6, label='Treated (A=1)', density=True)
axes[0].set_xlabel('Cognitive Test Score (Y)')
axes[0].set_ylabel('Density')
axes[0].set_title('Observed Outcome Distribution')
axes[0].legend()

# Plot 2: True treatment effects (heterogeneity)
true_te = Y1_true - Y0_true
axes[1].scatter(X[:, 0], true_te, alpha=0.4, s=20)
axes[1].axhline(y=true_te.mean(), color='r', linestyle='--', label=f'ATE = {true_te.mean():.2f}')
axes[1].set_xlabel('Covariate X₁ (Mother\'s Age)')
axes[1].set_ylabel('True Treatment Effect (Y¹ - Y⁰)')
axes[1].set_title('Heterogeneous Treatment Effects')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nAverage Treatment Effect (ATE): {true_te.mean():.2f}")
print(f"Treatment Effect Range: [{true_te.min():.2f}, {true_te.max():.2f}]")

## Interactive Configuration

Use the form below to configure the estimator. You can adjust:
- **Method**: ridge (kernel), df (deep features), or nk (neural-kernel)
- **Mode**: dr (doubly robust), onestep, ipw, or pi (plug-in)
- **Hyperparameters**: Model architecture and training settings

In [ ]:
# Create interactive configuration widgets

# Method and Mode selection
method_widget = widgets.Dropdown(
    options=['ridge', 'df', 'nk'],
    value='df',
    description='Method:',
    style={'description_width': '150px'}
)

mode_widget = widgets.Dropdown(
    options=['dr', 'onestep', 'ipw', 'pi'],
    value='dr',
    description='Mode:',
    style={'description_width': '150px'}
)

# Model hyperparameters
output_dim_widget = widgets.IntSlider(
    value=20,
    min=5,
    max=100,
    step=5,
    description='Number of Atoms/Features:',
    style={'description_width': '150px'}
)

hidden_dim_widget = widgets.Text(
    value='[20, 20]',
    description='Hidden Layers:',
    style={'description_width': '150px'},
    placeholder='[20, 20]'
)

sigma_widget = widgets.FloatSlider(
    value=2.0,
    min=0.1,
    max=10.0,
    step=0.1,
    description='Kernel Bandwidth (σ):',
    style={'description_width': '150px'}
)

lambda_widget = widgets.FloatText(
    value=20.0,
    description='Regularization (λ):',
    style={'description_width': '150px'}
)

# Training parameters
lr_or_widget = widgets.FloatText(
    value=2e-2,
    description='Learning Rate (Stage 1):',
    style={'description_width': '150px'}
)

lr_fi_widget = widgets.FloatText(
    value=2e-2,
    description='Learning Rate (Stage 2):',
    style={'description_width': '150px'}
)

epoch_or_widget = widgets.IntSlider(
    value=1000,
    min=100,
    max=10000,
    step=100,
    description='Epochs (Stage 1):',
    style={'description_width': '150px'}
)

epoch_fi_widget = widgets.IntSlider(
    value=500,
    min=100,
    max=5000,
    step=100,
    description='Epochs (Stage 2):',
    style={'description_width': '150px'}
)

batch_size_widget = widgets.IntSlider(
    value=128,
    min=32,
    max=512,
    step=32,
    description='Batch Size:',
    style={'description_width': '150px'}
)

# Validation and early stopping
valid_size_widget = widgets.FloatSlider(
    value=0.15,
    min=0.0,
    max=0.5,
    step=0.05,
    description='Validation Fraction:',
    style={'description_width': '150px'}
)

patience_widget = widgets.IntSlider(
    value=50,
    min=0,
    max=200,
    step=10,
    description='Early Stop Patience:',
    style={'description_width': '150px'}
)

seed_widget = widgets.IntText(
    value=42,
    description='Random Seed:',
    style={'description_width': '150px'}
)

verbose_widget = widgets.Checkbox(
    value=True,
    description='Verbose Training',
    style={'description_width': '150px'}
)

n_folds_widget = widgets.IntText(
    value=0,
    description='Cross-Fit Folds (0=off):',
    style={'description_width': '150px'}
)

# Organize widgets into sections
print("\n" + "="*60)
print("CONFIGURATION FORM")
print("="*60)

print("\n📊 METHOD & MODE")
display(method_widget, mode_widget)

print("\n🔧 MODEL HYPERPARAMETERS")
display(output_dim_widget, hidden_dim_widget, sigma_widget, lambda_widget)

print("\n🎓 TRAINING PARAMETERS")
display(lr_or_widget, lr_fi_widget, epoch_or_widget, epoch_fi_widget, batch_size_widget)

print("\n✅ VALIDATION & EARLY STOPPING")
display(valid_size_widget, patience_widget)

print("\n⚙️ OTHER SETTINGS")
display(seed_widget, verbose_widget, n_folds_widget)

print("\n" + "="*60)

## Build Configuration and Train Model

In [ ]:
# Build configuration from widgets
def build_config():
    """Build OmegaConf configuration from widget values"""
    
    # Parse hidden dimensions
    try:
        hidden_dim = eval(hidden_dim_widget.value)
        if not isinstance(hidden_dim, list):
            hidden_dim = [hidden_dim]
    except:
        hidden_dim = [20, 20]
        print("⚠️ Invalid hidden_dim format, using default [20, 20]")
    
    # Determine lambda value (null for nk, float for df/ridge)
    lamb_value = None if method_widget.value == 'nk' else lambda_widget.value
    
    # Validation size (0 means no validation)
    valid_size = valid_size_widget.value if valid_size_widget.value > 0 else None
    
    # Patience (0 means no early stopping)
    patience = patience_widget.value if patience_widget.value > 0 else None
    
    # Cross-fitting folds (0 means off / 50-50 split)
    n_folds = n_folds_widget.value if n_folds_widget.value >= 2 else None
    
    config_dict = {
        'method': method_widget.value,
        'mode': mode_widget.value,
        'seed': seed_widget.value,
        'verbose': verbose_widget.value,
        'n_folds': n_folds,
        'model': {
            'output_dim': output_dim_widget.value,
            'hidden_dim': hidden_dim,
            'sigma_init': sigma_widget.value,
            'lamb': lamb_value,
            'learn_sigma': False
        },
        'propensity_score_model': {
            '_target_': 'sklearn.ensemble.RandomForestClassifier',
            'max_depth': 4
        },
        'train': {
            'lr_or': lr_or_widget.value,
            'lr_fi': lr_fi_widget.value,
            'batch_size_or': batch_size_widget.value,
            'batch_size_fi': batch_size_widget.value,
            'epoch_or': epoch_or_widget.value,
            'epoch_fi': epoch_fi_widget.value,
            'valid_size_or': valid_size,
            'valid_size_fi': valid_size,
            'patience_or': patience,
            'patience_fi': patience
        },
        'test': {
            'num_bin': 1000
        }
    }
    
    return OmegaConf.create(config_dict)

# Build config
cfg = build_config()

print("\n✓ Configuration built successfully!")
print(f"\nMethod: {cfg.method} | Mode: {cfg.mode}")
print(f"Model: {cfg.model.output_dim} atoms/features, σ={cfg.model.sigma_init}, λ={cfg.model.lamb}")
print(f"Training: {cfg.train.epoch_or} epochs (stage 1), {cfg.train.epoch_fi} epochs (stage 2)")
if cfg.train.valid_size_or:
    print(f"Validation: {cfg.train.valid_size_or*100:.0f}% with patience={cfg.train.patience_or}")
if cfg.n_folds:
    print(f"Cross-fitting: {cfg.n_folds} folds")

In [ ]:
# Train the model
print("\n" + "="*60)
print("TRAINING CCME ESTIMATOR")
print("="*60 + "\n")

# Use first 5 covariates for conditioning
V = X[:, :5]

# Initialize and fit
estimator = CCDEstimator(cfg)
estimator.fit(X=X, V=V, Y=Y, A=A)

print("\n" + "="*60)
print("✓ TRAINING COMPLETED")
print("="*60)

## Visualize Estimated Densities

We'll visualize the estimated conditional counterfactual densities for different covariate profiles.

In [ ]:
# Select test points (different covariate profiles)
# Profile 1: Low mother's age/education
# Profile 2: High mother's age/education

V_test = np.array([
    [-1.5, -1.0, 0.0, 0.0, 0.0],  # Profile 1: Low
    [1.5, 1.0, 0.0, 0.0, 0.0]      # Profile 2: High
])

# Define grid for density estimation
y_min, y_max = Y.min() - 10, Y.max() + 10
Y_grid = np.linspace(y_min, y_max, 500)[:, None]

# Predict densities
pdf_est = estimator.predict(V_eval=V_test, Y_grid=Y_grid)

# Plot results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for i, (ax, profile_name) in enumerate(zip(axes, ['Low Covariate Profile', 'High Covariate Profile'])):
    ax.plot(Y_grid, pdf_est[i], linewidth=2.5, label=f'Estimated p(Y¹|V)', color='C0')
    ax.fill_between(Y_grid.flatten(), 0, pdf_est[i], alpha=0.3, color='C0')
    
    # Add vertical lines for mean and quantiles
    mean_est = np.sum(Y_grid.flatten() * pdf_est[i]) * (Y_grid[1] - Y_grid[0])
    ax.axvline(mean_est, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_est:.1f}')
    
    ax.set_xlabel('Outcome Y¹ (Cognitive Test Score)', fontsize=12)
    ax.set_ylabel('Density', fontsize=12)
    ax.set_title(f'{profile_name}\n{cfg.method}-{cfg.mode}', fontsize=13, fontweight='bold')
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Estimated Conditional Means:")
for i, profile in enumerate(['Low', 'High']):
    mean_est = np.sum(Y_grid.flatten() * pdf_est[i]) * (Y_grid[1] - Y_grid[0])
    print(f"  {profile} profile: E[Y¹|V] = {mean_est:.2f}")

## Distributional Insights Beyond Means

Extract richer information from the estimated densities:

In [ ]:
def compute_distributional_stats(y_grid, pdf):
    """Compute mean, variance, quantiles from density"""
    dy = y_grid[1] - y_grid[0]
    
    # Mean
    mean = np.sum(y_grid * pdf) * dy
    
    # Variance
    variance = np.sum((y_grid - mean)**2 * pdf) * dy
    std = np.sqrt(variance)
    
    # Quantiles
    cdf = np.cumsum(pdf) * dy
    q25 = y_grid[np.argmin(np.abs(cdf - 0.25))]
    q50 = y_grid[np.argmin(np.abs(cdf - 0.50))]
    q75 = y_grid[np.argmin(np.abs(cdf - 0.75))]
    
    return {
        'mean': mean,
        'std': std,
        'q25': q25,
        'q50': q50,
        'q75': q75
    }

# Compute stats for both profiles
y_grid_flat = Y_grid.flatten()

stats_low = compute_distributional_stats(y_grid_flat, pdf_est[0])
stats_high = compute_distributional_stats(y_grid_flat, pdf_est[1])

# Display results
results_df = pd.DataFrame({
    'Low Profile': [f"{stats_low['mean']:.2f}", f"{stats_low['std']:.2f}", 
                    f"{stats_low['q25']:.2f}", f"{stats_low['q50']:.2f}", f"{stats_low['q75']:.2f}"],
    'High Profile': [f"{stats_high['mean']:.2f}", f"{stats_high['std']:.2f}",
                     f"{stats_high['q25']:.2f}", f"{stats_high['q50']:.2f}", f"{stats_high['q75']:.2f}"]
}, index=['Mean', 'Std Dev', '25th Percentile', 'Median', '75th Percentile'])

print("\n📊 DISTRIBUTIONAL STATISTICS")
print("="*60)
print(results_df)
print("="*60)

# Interpretation
mean_diff = stats_high['mean'] - stats_low['mean']
print(f"\n💡 INSIGHTS:")
print(f"  • Conditional Average Treatment Effect difference: {mean_diff:.2f}")
print(f"  • High profile has {'higher' if mean_diff > 0 else 'lower'} expected outcomes")
print(f"  • Low profile uncertainty (std): {stats_low['std']:.2f}")
print(f"  • High profile uncertainty (std): {stats_high['std']:.2f}")
print(f"  • IQR (Low): {stats_low['q75'] - stats_low['q25']:.2f}")
print(f"  • IQR (High): {stats_high['q75'] - stats_high['q25']:.2f}")

## Compare Multiple Methods (Optional)

Compare CME, DF, and NN estimators on the same profile:

In [ ]:
# Train all three methods
methods = ['ridge', 'df', 'nk']
colors = ['C0', 'C1', 'C2']
estimators = {}
predictions = {}

print("\nTraining multiple methods for comparison...\n")

for method in methods:
    print(f"Training {method}...")
    
    # Update config
    cfg_temp = OmegaConf.create(OmegaConf.to_container(cfg))
    cfg_temp.method = method
    
    # Adjust hyperparameters by method
    if method == 'ridge':
        cfg_temp.model.lamb = 20.0
        cfg_temp.train.epoch_or = 1
        cfg_temp.train.epoch_fi = 1
    elif method == 'df':
        cfg_temp.model.lamb = 20.0
        cfg_temp.train.epoch_or = 1000
        cfg_temp.train.epoch_fi = 500
    else:  # nk
        cfg_temp.model.lamb = None
        cfg_temp.train.epoch_or = 1000
        cfg_temp.train.epoch_fi = 500
    
    cfg_temp.verbose = False  # Suppress output for cleaner comparison
    
    # Train
    est = CCDEstimator(cfg_temp)
    est.fit(X=X, V=V, Y=Y, A=A)
    
    # Predict on first profile
    pred = est.predict(V_eval=V_test[:1], Y_grid=Y_grid)
    
    estimators[method] = est
    predictions[method] = pred[0]

print("\n✓ All methods trained!\n")

# Plot comparison
plt.figure(figsize=(10, 6))

for method, color in zip(methods, colors):
    plt.plot(Y_grid, predictions[method], linewidth=2.5, label=f'{method}-{cfg.mode}', color=color, alpha=0.8)

plt.xlabel('Outcome Y¹ (Cognitive Test Score)', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.title(f'Method Comparison: Estimated p(Y¹|V)\nLow Covariate Profile', fontsize=13, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n📊 Estimated Means by Method:")
for method in methods:
    mean_est = np.sum(y_grid_flat * predictions[method]) * (Y_grid[1] - Y_grid[0])
    print(f"  {method}: E[Y¹|V] = {mean_est:.2f}")

## Summary

This notebook demonstrated:

1. ✅ **Interactive Configuration**: Use widgets instead of YAML files
2. ✅ **Conditional Density Estimation**: Estimate $p(Y^1|V)$ for different covariate profiles
3. ✅ **Distributional Insights**: Extract means, variances, quantiles beyond ATEs
4. ✅ **Method Comparison**: Compare ridge, df, and nk estimators
5. ✅ **Heterogeneous Effects**: Visualize how treatment effects vary across populations

### Next Steps:

- Try different methods (ridge, df, nk) and modes (dr, onestep)
- Adjust hyperparameters to see their effects
- Use real IHDP data for more realistic results
- Explore other datasets (LaLonde, 401k, etc.)

### Key Takeaways:

- **Beyond means**: Full densities reveal heterogeneity hidden by average effects
- **Double robustness**: dr mode provides protection against model misspecification
- **Flexible estimation**: Neural methods (df, nk) scale to complex settings